Import Everything

In [ ]:
import pandas as pd
import yaml
import numpy as np
import obspy
from obspy import UTCDateTime
from time import time
import csv
from glob import glob
import re

In [ ]:
with open('/home/smocz/expand_redpy/scripts/config.yaml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

vv = config['vv']
volc_list = ['Baker','Hood','Newberry','Rainier','St_Helens']
volc = volc_list[vv]
year = config['year']

homedir = config['homedir']
readdir = config['readdir']

#time in seconds before and after a detection to check for similar detections
wi = config['wi']

Parameters and dataframes for each station

In [ ]:
#get volcano metadata
df = pd.read_csv(readdir+'Volcano_Metadata.csv')
#station list
sta_list = df[df['Volcano_Name']==volc]['Station'].values.tolist() #CHANGE MT_BAKER TO VOLC
#panda dataframe for detections at a station
readstadict = {}
for i in sta_list:
    readstadict[i] = pd.read_csv(homedir+'detections/'+volc+'_'+i+'_'+str(year)+'_clean_detections.csv') 
rsta_list = []
for i in readstadict:
    rsta_list.append(readstadict[i])
readsta = pd.concat(rsta_list)
print(sta_list)
print(type(rsta_list[0]))
print(rsta_list)
print(readsta)

# NOTE FOR EDITING: be sure to also update below cell when adding stations
### new loop per station, update sta#times and readsta#, update match#, and add match# to match_list

In [ ]:
t0 = time()
with open(homedir+'events/'+volc+'_'+year+'_events.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Earliest_Detection_Time","Cluster_ID","Stations_Found","Stations"])
    #detection time, cluster id (no longer needs to be separated by station), and number of stations with a 
    #detection for this event
    file.close()

#for all detections we want to run (concatenated)
temp_name_list = readsta['Template_Name'].values.tolist() #make a list of template names
cl_list_long = [] # make a list of the numbers in each template name
for i in temp_name_list: 
    num = re.findall(r'\d+', i)
    cl_list_long.append(*num)
cl_list = np.unique(cl_list_long) #get rid of duplicates

#for cluster x
for cl in cl_list:
    t1 = time()
#for detection x
    times = []
    for i in np.unique(temp_name_list):
        if i.endswith(cl):
            all_times = readsta[readsta['Template_Name']==i]['Detection_Time'].values.tolist()
            for at in all_times:
                times.append(at)
    for ii,i in enumerate(times):
        t3 = time()
#run through each detection time on all stations
        print( )
        print('------------')
        print('trying detection',ii,'cluster',cl,i)
        
        match_list = []
        for ss,s in enumerate(sta_list):
            statimes = []
            for a in np.unique(readstadict[s]['Template_Name'].values.tolist()):
                if a.endswith(cl):
                    all_times = readstadict[s][readstadict[s]['Template_Name']==a]['Detection_Time'].values.tolist()
                    for at in all_times:
                        statimes.append(at)
            match=0 #set variable to arbitrary number
            for tt,t in enumerate(statimes):
                ts = UTCDateTime(t)-wi
                te = UTCDateTime(t)+wi
                if UTCDateTime(i)>ts and UTCDateTime(i)<te:
                    match=2 #if there is an overlap, reset the variable and break out of the loop
                    print('Overlap with station '+s+' detections')
                match_list.append(match)
        print('match_list:',match_list)
        
        t2 = time()
        print(t2-t3,'seconds to test overlap for detection',ii)
    
        save_list = [] #for saving a list of stations
        for mm,m in enumerate(match_list):
            if m==2:
                save_list.append(sta_list[mm])

        if match_list.count(2) >= 4: #if at least 4 matches equal 2:
#         if match==2 and match0==2 and match1==2 and match2==2:
            print('saving...')
            #ADD A CATCH: if an event of this cluster and this time give or take 1 second 
            #already exists, do NOT save
            check = pd.read_csv(homedir+'events/'+volc+'_'+str(year)+'_events.csv')
            checktimes = check[check['Cluster_ID']==int(cl)]['Detection_Time'].values.tolist()
            checking = 0
            for tt,t in enumerate(checktimes):
                ts = UTCDateTime(t)-wi
                te = UTCDateTime(t)+wi
                if UTCDateTime(i)>ts and UTCDateTime(i)<te: 
                    print('already recorded event')
                    checking=1 #checking still =1 because we don't want the normal save process
                    #if i is earlier (less than) than the recorded event, overwrite the event (we want the earliest detected time)
                    if i<t:
                        print('replacing...')
                        print('old time:',t,'new time:',i)
                        check.replace(t,i,inplace=True) #ONLY WORKS FOR THE DATAFRAME NOT THE CSV
                        check.to_csv(homedir+'events/'+volc+'_'+year+'_events.csv',index=False)
                        #read panda dataframe - change dataframe, overwrite the csv with this dataframe
                        print('changed csv')
                        #2+ clusters at the same time shouldn't happen because templates are 13s long, with 
                        #trig int of 6. any '2' events in the same window would just be one event and waves 
                        #shouldn't be so slow that a different one can be at the exact time (within 1 second 
                        #of the detection)
                    break
            if checking==1: continue
            row = [i,int(cl),match_list.count(2),' '.join(save_list)]
            print(row)
            
            with open(homedir+'events/'+volc+'_'+year+'_events.csv', 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(row)
                file.close()
    t4 = time()
    print(t4-t3,'seconds for cluster',cl)
t5 = time()
print(t5-t0,'seconds to find all new events on',volc,'in',year)